#INTENTION OF THE PROJECT

Hello!!
The purpose of this project is to :

*   Do the indepth analysis between two classifiers.
  1.   Multinomial Naive Bayes
  2.   Logistic Reg.

* Understand the type of data with given using visualization using given information and data file.

* Build the classifier that predits the rating for the given review.

I have selected Logistic Regression since it appropriate regression analysis to conduct when there are diffrent variables involved. Like all regression analyses, the logistic regression is a predictive analysis.  Logistic regression is used to describe data and to explain the relationship between one dependent binary variable and one or more nominal, ordinal, interval or ratio-level independent variables.

To test this, I have used Multinomial Naive Bayes since it is usually used in many Natural Language Processing tasks.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from string import digits
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import svm, linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.ensemble import VotingClassifier 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import random
from sklearn.metrics import accuracy_score

In [ ]:
data= pd.read_csv('../input/boardgamegeek-reviews/bgg-13m-reviews.csv')
infofile= pd.read_csv('../input/boardgamegeek-reviews/games_detailed_info.csv')

We have two files 
1. bgg-13m-reviews.csv
2. games_detailed_info.csv

The first file is the data that contains 13 million reviews. Where we will build our model , do the analysis, and later use for do live prediction.

The second file has the aggregated deatils about the data, which will help us do the visualizations. 

In [ ]:
Now, We will check the data by doing following.
head() call on object returns the first 5 rows.

In [ ]:
data.head()

In [ ]:
infofile.head()

VISUALS

Next we can see how the ratings are distributed for all the games and number of copies that are owned by all reviewers. This way we know that the data is mostly the reviews/rating by people who own the game.


We can also see, Catan is most favourite boardgame ever created. We have many such games in the list as outliers.
We can study what made these games so popular, and use those parameters to predict the particular game-publishers game-success rate.

In [ ]:
#Set a style sheet and instantiate fig and ax
plt.style.use('fivethirtyeight')
fig,ax = plt.subplots()

#set our basic labels
ax.set_title(label='Top 1% of Games by \nRating and Users Who Own', 
             fontsize=16, fontweight='bold')
ax.set_xlabel('User Rating', fontsize=12)
ax.set_ylabel('# of Copies Owned', fontsize=12)

#we will label some outliers
ax.text(x=6.75, y=122000, s='Catan', fontsize=10)
ax.text(x=7.00, y=115500, s='Carcassone', fontsize=10)
ax.text(x=7.60, y=121000, s='Pandemic', fontsize=10)
ax.text(x=8.35, y=31500, s='Gloomhaven', fontsize=10)

#Edit tick details
ax.set(yticks=range(0,130000,25000))
ax.tick_params(axis='both', which='major', labelsize=10)

#plot the data
plt.scatter(x=infofile['bayesaverage'],
            y=infofile['owned'],
            s=infofile['averageweight']**4, color='blue', alpha=.5, label='Top Rated')
plt.scatter(x=infofile['bayesaverage'],
            y=infofile['owned'],
            s=infofile['averageweight']**4,color = 'red', 
            alpha=.3, label='Top Owned');

#Create and tweak details of the legend
lgnd = plt.legend(loc=(.1,.2),scatterpoints=1, fontsize=10, 
                  title_fontsize=10, title='Size by Weight')
lgnd.legendHandles[0]._sizes = [100]
lgnd.legendHandles[1]._sizes = [210]

#Save
plt.savefig('Games_by_rating_and_owned.png', format='png', 
            dpi=300, bbox_inches='tight')

**CLEANING**

Next we will do th cleaning on the data by removing NOT REQUIRED columns.

In [ ]:
data1 = data.drop(columns="ID")
data2 = data1.drop(columns="name")
data3 = data2.drop(columns="user")
data3.head()

Next, we will remove the NULL reviews from the comment column.

In [ ]:
dataset = data3.dropna(subset=['comment'])
dataset.head()

Now its time to removing all the punctuation and making all the letter lower case

In [ ]:
 dataset['comments'] = dataset['comment'].str.lower().apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
 dataset = dataset.drop(columns="comment")
 dataset.head()

Now we will try to remove all the stopwords from english lauguage and try to add few more stopwords that we think are redundant to the dataset.

In [ ]:
stpwrds = stopwords.words('english')
stpwrds.extend(('game','people','really','board','games','play','player','one','plays','cards','would','played','players'))
dataset['comments'] = dataset['comments'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stpwrds)]))
dataset.head()

Next, we will work on the rating column.
There is the rounding up of values that we will do

In [ ]:
dataset['rating'] = dataset['rating'].round() 
dataset.head()

**VISUALS (ON CLEANED DATA)**

This is negavtive review word cloud of the data

In [ ]:
negative = dataset.loc[dataset['rating'] < 2]
words = Counter([comm for comm in " ".join(negative['comments']).split()])
wc = WordCloud(width=400, height=350,colormap='plasma',background_color='white').generate_from_frequencies(dict(words.most_common(200)))
plt.figure(figsize=(15,10))
plt.imshow(wc, interpolation='bilinear')
plt.title('Common Words in Negative Reviews', fontsize=20)
plt.axis('off');
plt.show()

This is positive review word cloud of the data.

In [ ]:
positive = dataset.loc[dataset['rating'] > 9]
words = Counter([w for w in " ".join(positive['comments']).split()])
wc = WordCloud(width=400, height=350,colormap='plasma',background_color='white').generate_from_frequencies(dict(words.most_common(200)))
plt.figure(figsize=(15,10))
plt.imshow(wc, interpolation='bilinear')
plt.title('Common Words in Positive Reviews', fontsize=20)
plt.axis('off');
plt.show();

**DATA SPLIT**

Now that our data is ready to be processed upon, we will split the data into 70% of traning set and 30% of test set.

In [ ]:
train_data, test_data = train_test_split(dataset, test_size = 0.30, random_state= 2)

Training data look:

In [ ]:
train_data.head()

Test data look:

In [ ]:
test_data.head()

Now, for to facilitate the execution of the code and handling the data efficiently, we further reduce the training data into two intervals:
1. inter1
2. inter2

In [ ]:
inter1, inter2 = train_test_split(train_data, test_size = 50, random_state= 2)

**MODEL SELECTION AND TRAINING:**

First, We will use the interval1 of training set to see the accuracy of the model. We have selected Multinomial naive bayes it estimates the conditional probability of a particular word given a class as the relative frequency of term t in documents belonging to class. The variation takes into account the number of occurrences of term t in training documents from class,including multiple occurrences.

Following this code, is the accuracy of this model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(inter1.comments, inter1.rating, random_state=2,test_size=0.30)

model1 = Pipeline([
    ('count_vectorizer', CountVectorizer()), ('tfidf_transformer',  TfidfTransformer()),('classifier', MultinomialNB()) ])

    
model1.fit(X_train,y_train.astype('int'))

labels = model1.predict(X_test)

accr = accuracy_score(y_test.astype('int'),labels, normalize=True) * float(100)
print('ACCURACY OF MULTINOMIAL NAIVE BAYES FOR FIRST PART OF TRAIN_SET IS',(accr))

Next, we will try Logistic regression. It is the appropriate regression analysis to conduct when the dependent variable is dichotomous (binary).  Like all regression analyses, the logistic regression is a predictive analysis.  Logistic regression is used to describe data and to explain the relationship between one dependent binary variable and one or more nominal, ordinal, interval or ratio-level independent variables.


Following this code, is the accuracy of this model.

In [ ]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(inter1.comments, inter1.rating, random_state=2,test_size=0.30)

model2 = Pipeline([
    ('count_vectorizer', CountVectorizer()), ('tfidf_transformer',  TfidfTransformer()),('classifier', LogisticRegression()) ])

    
model2.fit(X_train,y_train.astype('int'))

labels = model2.predict(X_test)

accr = accuracy_score(y_test.astype('int'),labels, normalize=True) * float(100)
print('ACCURACY OF LOGISTIC REG. FOR FIRST PART OF TRAIN_SET IS',(accr))


From the above two models and the comparision between their performance and accuracy we can see, 
Logistic reg is performing better than Multinomial Naive bayes.

We will try the same models on different dataset to reassure this hypothesis.

Following this code, is the accuracy of this model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(inter2.comments, inter2.rating, random_state=2,test_size=0.30)

model3 = Pipeline([
    ('count_vectorizer', CountVectorizer()), ('tfidf_transformer',  TfidfTransformer()), ('classifier', MultinomialNB()) ])

    
model3.fit(X_train,y_train.astype('int'))

labels = model3.predict(X_test)

accr = accuracy_score(y_test.astype('int'),labels, normalize=True) * float(100)
print('ACCURACY OF MULTINOMIAL NAIVE BAYES FOR SECOND PART OF TRAIN_SET IS',(accr))

Finally, 
we will again use the logistic reg. to compare them with above results.

Following this code, is the accuracy of this model.

In [ ]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(inter2.comments, inter2.rating, random_state=2,test_size=0.30)

model4 = Pipeline([
    ('count_vectorizer', CountVectorizer()), ('tfidf_transformer',  TfidfTransformer()), ('classifier', LogisticRegression()) ])

    
model4.fit(X_train,y_train.astype('int'))

labels = model4.predict(X_test)

accr = accuracy_score(y_test.astype('int'),labels, normalize=True) * float(100)
print('ACCURACY OF LOGISTIC REG. FOR SECOND PART OF TRAIN_SET IS',(accr))

**MY CONTRIBUTION FROM THE ANALYSIS WE MADE.**




After building and running these two very efficient models on the different type of data,
We can see how logistic reg. performed better than Multinomial NB.

To conculude this, 
We will run this model on entire training data, 
so the learning becomes versatile and prediction becomes more accurate in the end.

In [ ]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(train_data.comments, train_data.rating, random_state=2,test_size=0.30)

model_final = Pipeline([
    ('count_vectorizer', CountVectorizer()),('tfidf_transformer',  TfidfTransformer()), ('classifier', LogisticRegression()) ])

model_final.fit(X_train,y_train.astype('int'))

labels = model_final.predict(X_test)

accr = accuracy_score(y_test.astype('int'),labels, normalize=True) * float(100)
print('ACCURACY OF OUR FINAL MODEL OF TRAIN_SET IS',(accr))

**CONCLUSION:**
Finally after concluding the final model for this game review data,
We are going to deploy to start predicting the live reviews for our customers.








**THANK YOU**,

Sohan Badade

ID 1001729097

References:

Reference Links
https://github.com/jushih/Sentiment-Analysis

http://www.site.uottawa.ca/~stan/csi5387/DMNB-paper.pdf

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

https://scikit-learn.org/stable/modules/svm.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html

https://monkeylearn.com/text-classification/

https://www.geeksforgeeks.org/applying-multinomial-naive-bayes-to-nlp-problems/

